In [1]:
from langchain.document_loaders import PyPDFDirectoryLoader

directory = 'C:/Users/91630/Desktop/Project-P2/Documents'

def load_docs(directory):
  loader = PyPDFDirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)
documents


[Document(page_content='About Brightspeed:\nBrightspeed is a telecommunications company focused on ensuring people have \nreliable internet access. Our mission is to provide high-quality internet \nservices to the communities we serve, helping individuals stay connected for \nwork, learning, entertainment, and more.\nFormed in August 2021 through a collaboration between Connect Holding, LLC and \nLumen Technologies, Inc., Brightspeed operates in 20 states, mainly in the \nMidwest and Southeast regions of the United States. We offer both internet and \nvoice services to homes and businesses, aiming to keep our customers connected \nand productive.\nBrightspeed’s headquarters: Brightspeed is located in Charlotte, North Carolina.\nWhy Brightspeed:\nBrightspeed stands out for its commitment to simplicity, transparency, and \ncustomer satisfaction:\n1. Simplified Experience: No hidden fees, straightforward billing, and no \ncontracts.\n2. Reliable Connectivity: Affordable, dependable intern

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=500,chunk_overlap=20):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))


255


In [3]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-large",
encode_kwargs={"normalize_embeddings": True},)

c:\Users\91630\pyfolder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pinecone 
from langchain.vectorstores import Pinecone

# Create an instance of the Pinecone class
pc = pinecone.Pinecone(
    api_key="PINECONE_API_KEY",  # find at app.pinecone.io
    environment="gcp-starter"  # next to api key in console
)

# Now do stuff
index_name = "gen-ai"
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)


In [ ]:
def get_similiar_docs(query,k=1):
  similar_docs = index.similarity_search(query,k=k)
  return similar_docs


In [ ]:
%%time
query="What is brightspeed future plans?"
results=get_similiar_docs(query)
results

CPU times: total: 953 ms
Wall time: 1.17 s


[Document(page_content='Brightspeed â  Born from Lumen Technologiesâ  sale of ILEC assets in 20 states \x80\x93 \x80\x99\nto Apollo Management, Brightspeed is looking to reach more than 1 million \npassings by the end of 2023 and 3 million within five years. Its 2023 plans \ninclude 300,000 new passings in North Carolina, 170,000 locations in Ohio, \n130,000 passings in Missouri and 120,000 in Texas.\nFrontier Communications â  After emerging from bankruptcy in 2021, Frontier \x80\x93', metadata={'page': 15.0, 'source': 'C:\\Users\\91630\\Desktop\\Project-P2\\Documents\\Brightspeed_data.pdf'})]

In [ ]:
from langchain.llms.huggingface_hub import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature":0.0, "max_length":200}
)

In [ ]:
from langchain.prompts.prompt import PromptTemplate


template_string="""
<system>\
You are a honest brightspeed agent and your job is giving answers based on content. If user ask any question is not related to content then say "I don't know, Please ask anything about Brightspeed". The answer should be in simple statement.
</system>
###
content: {content}
####
<user>{text}</user>
###
answer:
"""


prompt = PromptTemplate.from_template(template=template_string)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [ ]:
%%time
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    output_parser=output_parser
)

res=chain.invoke({"content":results,"text":query})
res['text']

CPU times: total: 750 ms
Wall time: 49.8 s


'Brightspeed is looking to reach more than 1 million passings by the end of 2023 and 3 million within five years.'